<a href="https://colab.research.google.com/github/sanskriti49/Sentimental-Analysis/blob/main/Sentiment_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importing libraries

In [33]:
!git clone https://github.com/sanskriti49/Sentimental-Analysis
import numpy as np
import pandas as pd

fatal: destination path 'Sentimental-Analysis' already exists and is not an empty directory.


###Importing dataset

In [34]:
url="https://raw.githubusercontent.com/sanskriti49/Sentimental-Analysis/refs/heads/main/data/a2_RestaurantReviews_FreshDump.tsv"
dataset = pd.read_csv(url, delimiter = '\t', quoting = 3)
dataset.head()

,Review
0,Spend your money elsewhere.
1,Their regular toasted bread was equally satisf...
2,The Buffet at Bellagio was far from what I ant...
3,"And the drinks are WEAK, people!"
4,-My order was not correct.


###Data cleaning

In [35]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
corpus=[]

for i in range(0, len(dataset)):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

###Data transformation

In [37]:
# Loading BoW dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile = '/content/Sentimental-Analysis/models/c1_BoW_Sentiment_Model.pkl'
cv = pickle.load(open(cvFile, "rb"))


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [38]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(100, 1420)

###Predictions (via sentiment classifier)

In [39]:
import joblib
classifier = joblib.load('/content/Sentimental-Analysis/models/c2_Classifier_Sentiment_Model')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator GaussianNB from version 1.0.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [40]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

#Mapping predicted labels to sentiment labels
sentiments=["Negative" if label==0 else "Positive" for label in y_pred]

[0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0
 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


In [41]:
# dataset['predicted_label'] = y_pred.tolist()
dataset['predicted_label'] =sentiments
dataset.head()

,Review,predicted_label
0,Spend your money elsewhere.,Negative
1,Their regular toasted bread was equally satisf...,Positive
2,The Buffet at Bellagio was far from what I ant...,Positive
3,"And the drinks are WEAK, people!",Negative
4,-My order was not correct.,Negative


In [42]:
dataset.to_csv("c3_Predicted_Sentiments_Fresh_Dump.tsv", sep='\t', encoding='UTF-8', index=False)